# Модуль Б. Анализ и визуализация данных

## Цели модуля:
1. **Построение аналитической системы** - дашборд Streamlit
2. **Анализ трендов** - региональная и сезонная активность
3. **Разметка опасных участков** - кластеризация и классификация рисков
4. **Оценка качества разметки** - сравнение методов
5. **Подготовка отчета** и сохранение размеченных данных

## Входные данные:
- `final_dataset.csv` из Модуля А
- или `tracks.db` (SQLite база)

## Выходные данные:
- `labeled_dataset.csv` - размеченные данные с уровнями риска
- Дашборд Streamlit (`app.py`)
- Визуализации и отчеты


In [ ]:
# ===== Часть 0: Импорт библиотек и загрузка данных =====
import os
from dotenv import load_dotenv

# Загружаем переменные окружения
load_dotenv()

print("📁 Переменные окружения загружены")

# ===== Часть 0: Импорт и загрузка данных (исправлено) =====
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium
from folium.plugins import HeatMap
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')
import json  # Добавлено

pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📊 МОДУЛЬ Б: Анализ и визуализация данных")
print("=" * 60)

# --- Настройки загрузки из .env ---
DATA_SOURCES = []

# Приоритет источников
priority = os.getenv("DATA_SOURCE_PRIORITY", "csv,db").split(",")

for src_type in priority:
    if src_type.strip() == "csv":
        csv1 = os.getenv("CSV_PATH_1", "labeled_dataset.csv")
        csv2 = os.getenv("CSV_PATH_2", "../module_a/final_dataset.csv")
        DATA_SOURCES.extend([csv1, csv2])
    elif src_type.strip() == "db":
        db1 = os.getenv("DB_PATH", "tracks.db")
        db2 = os.getenv("DB_PATH_FALLBACK", "../module_a/tracks.db")
        DATA_SOURCES.extend([db1, db2])

# --- Загрузка ---
df = None
for source in DATA_SOURCES:
    try:
        if source.endswith('.csv') and os.path.exists(source):
            df = pd.read_csv(source, encoding='utf-8')
            print(f"✅ Данные загружены из CSV: {source}")
            break
        elif source.endswith('.db') and os.path.exists(source):
            conn = sqlite3.connect(source)
            df_tracks = pd.read_sql('SELECT * FROM tracks', conn)
            df_points = pd.read_sql('SELECT * FROM track_points', conn)
            conn.close()
            df = pd.merge(df_points, df_tracks, on='track_id', how='left')
            print(f"✅ Данные загружены из БД: {source}")
            break
    except Exception as e:
        print(f"❌ Ошибка при загрузке из {source}: {e}")

if df is None:
    print("⚠️ Создаю тестовые данные...")
    np.random.seed(42)
    n_points = 1000
    df = pd.DataFrame({
        'track_id': [f'track_{i//10}' for i in range(n_points)],
        'date': pd.date_range('2024-01-01', periods=n_points, freq='H').strftime('%Y-%m-%d'),
        'region': np.random.choice(['Москва', 'Кавказ', 'Урал', 'Сибирь'], n_points),
        'lat': 55.75 + np.random.randn(n_points) * 0.1,
        'lon': 37.62 + np.random.randn(n_points) * 0.1,
        'step_frequency': np.random.normal(90, 15, n_points),
        'elevation': np.random.uniform(50, 1000, n_points),
        'temperature': np.random.uniform(-10, 25, n_points),
        'terrain_type': np.random.choice(['горы', 'холмы', 'равнина', 'низменность'], n_points),
        'season': np.random.choice(['зима', 'весна', 'лето', 'осень'], n_points),
        'point_index': list(range(n_points))
    })

print(f"📈 Размер датасета: {df.shape}")
print(f"📅 Диапазон дат: {df['date'].min()} - {df['date'].max()}")
print(f"🌍 Регионы: {df['region'].unique().tolist()}")
print("-" * 60)
display(df.head(3))

In [ ]:
# ===== Часть 1: Анализ трендов и зависимостей =====
print("📊 АНАЛИЗ ТРЕНДОВ И ЗАВИСИМОСТЕЙ")
print("=" * 60)

# Создаем словарь для хранения результатов анализа
analysis_results = {}

try:
    # 1.1 Средняя частота шагов по сезонам
    if 'season' in df.columns and 'step_frequency' in df.columns:
        print("🔍 Анализ сезонных трендов...")
        season_stats = df.groupby('season')['step_frequency'].agg(['mean', 'std', 'count']).reset_index()
        season_stats = season_stats.sort_values('mean', ascending=False)
        
        # Сохраняем для отчета
        analysis_results['season_stats'] = season_stats.to_dict('records')
        
        # Создаем визуализацию
        fig = px.bar(
            season_stats, 
            x='season', 
            y='mean', 
            error_y='std',
            title='Средняя частота шагов по сезонам',
            labels={'mean': 'Средняя частота шагов (шаг/мин)', 'season': 'Сезон'},
            color='mean',
            color_continuous_scale='Viridis',
            text='mean'
        )
        fig.update_traces(texttemplate='%{text:.1f}', textposition='outside')
        fig.update_layout(
            showlegend=False,
            yaxis_title="Частота шагов (шаг/мин)",
            xaxis_title="Сезон",
            height=500
        )
        fig.show()
        
        print("\n📈 Статистика по сезонам:")
        display(season_stats.style.format({'mean': '{:.1f}', 'std': '{:.1f}'}))
    else:
        print("⚠️ Отсутствуют данные о сезонах или частоте шагов")

    # 1.2 Зависимость температуры от времени суток (на основе даты)
    if 'temperature' in df.columns:
        print("\n🌡️ Анализ температурных трендов...")
        try:
            # Преобразуем дату и извлекаем час
            df_temp = df.copy()
            df_temp['hour'] = pd.to_datetime(df_temp['date'], errors='coerce').dt.hour
            
            # Группируем по часу
            temp_by_hour = df_temp.groupby('hour')['temperature'].agg(['mean', 'std', 'count']).reset_index()
            temp_by_hour = temp_by_hour.dropna()
            
            if not temp_by_hour.empty:
                analysis_results['temp_by_hour'] = temp_by_hour.to_dict('records')
                
                # Создаем график
                fig = px.line(
                    temp_by_hour, 
                    x='hour', 
                    y='mean',
                    title='Зависимость температуры от времени суток',
                    labels={'mean': 'Средняя температура (°C)', 'hour': 'Час суток'},
                    markers=True
                )
                
                # Добавляем доверительный интервал
                fig.add_scatter(
                    x=temp_by_hour['hour'], 
                    y=temp_by_hour['mean'] + temp_by_hour['std'],
                    mode='lines',
                    line=dict(width=0),
                    showlegend=False,
                    name='+1 std'
                )
                fig.add_scatter(
                    x=temp_by_hour['hour'], 
                    y=temp_by_hour['mean'] - temp_by_hour['std'],
                    mode='lines',
                    fill='tonexty',
                    line=dict(width=0),
                    fillcolor='rgba(100, 100, 200, 0.2)',
                    showlegend=False,
                    name='-1 std'
                )
                
                fig.update_layout(
                    xaxis=dict(tickmode='linear', dtick=2),
                    yaxis_title="Температура (°C)",
                    xaxis_title="Час суток",
                    height=500
                )
                fig.show()
                
                print("✅ Температурный анализ завершен")
            else:
                print("⚠️ Недостаточно данных для температурного анализа")
        except Exception as e:
            print(f"⚠️ Ошибка при анализе температуры: {e}")
    else:
        print("⚠️ Отсутствуют данные о температуре")

    # 1.3 Влияние типа местности на активность туристов
    if 'terrain_type' in df.columns and 'step_frequency' in df.columns:
        print("\n🏞️ Анализ влияния местности...")
        terrain_effect = df.groupby('terrain_type')['step_frequency'].agg(['mean', 'count']).reset_index()
        terrain_effect = terrain_effect.sort_values('mean', ascending=False)
        
        analysis_results['terrain_effect'] = terrain_effect.to_dict('records')
        
        # Создаем комбинированную визуализацию
        fig = make_subplots(
            rows=1, 
            cols=2, 
            subplot_titles=('Средняя частота шагов по типам местности', 'Распределение типов местности'),
            specs=[[{'type': 'bar'}, {'type': 'pie'}]],
            horizontal_spacing=0.2
        )
        
        # Бар-чарт
        fig.add_trace(
            go.Bar(
                x=terrain_effect['terrain_type'], 
                y=terrain_effect['mean'],
                name='Средняя частота шагов',
                marker_color='coral',
                text=terrain_effect['mean'].round(1),
                textposition='outside'
            ),
            row=1, col=1
        )
        
        # Круговая диаграмма
        fig.add_trace(
            go.Pie(
                labels=terrain_effect['terrain_type'], 
                values=terrain_effect['count'],
                name='Распределение',
                hole=0.3,
                textinfo='percent+label'
            ),
            row=1, col=2
        )
        
        fig.update_layout(
            title_text='Влияние типа местности на активность туристов', 
            height=500,
            showlegend=False
        )
        fig.update_xaxes(title_text="Тип местности", row=1, col=1)
        fig.update_yaxes(title_text="Частота шагов (шаг/мин)", row=1, col=1)
        
        fig.show()
        
        print("\n📊 Влияние типа местности:")
        display(terrain_effect.style.format({'mean': '{:.1f}'}))
    else:
        print("⚠️ Отсутствуют данные о типе местности или частоте шагов")

    # 1.4 Связь высоты маршрута с частотой шагов
    if 'elevation' in df.columns and 'step_frequency' in df.columns:
        print("\n⛰️ Анализ зависимости от высоты...")
        
        # Удаляем выбросы
        df_elev = df.copy()
        q1 = df_elev['elevation'].quantile(0.05)
        q3 = df_elev['elevation'].quantile(0.95)
        df_elev = df_elev[(df_elev['elevation'] >= q1) & (df_elev['elevation'] <= q3)]
        
        # Бинаризация высоты
        df_elev['elevation_bin'] = pd.cut(
            df_elev['elevation'], 
            bins=min(10, len(df_elev) // 20),  # Динамическое количество бинов
            precision=0
        )
        
        elevation_stats = df_elev.groupby('elevation_bin')['step_frequency'].agg(['mean', 'std', 'count']).reset_index()
        elevation_stats['mid'] = elevation_stats['elevation_bin'].apply(lambda x: x.mid)
        
        analysis_results['elevation_stats'] = elevation_stats.to_dict('records')
        
        # Создаем scatter plot с трендом
        fig = px.scatter(
            df_elev, 
            x='elevation', 
            y='step_frequency', 
            trendline='lowess',
            trendline_color_override='red',
            title='Зависимость частоты шагов от высоты',
            labels={'elevation': 'Высота (м)', 'step_frequency': 'Частота шагов (шаг/мин)'},
            opacity=0.5,
            color_discrete_sequence=['blue']
        )
        
        # Добавляем средние по бинам
        fig.add_trace(
            go.Scatter(
                x=elevation_stats['mid'], 
                y=elevation_stats['mean'],
                mode='lines+markers',
                name='Среднее по бинам',
                line=dict(color='green', width=3),
                marker=dict(size=8, color='green')
            )
        )
        
        fig.update_layout(
            height=500,
            xaxis_title="Высота над уровнем моря (м)",
            yaxis_title="Частота шагов (шаг/мин)"
        )
        
        fig.show()
        
        # Корреляция
        corr = df_elev[['elevation', 'step_frequency']].corr().iloc[0, 1]
        analysis_results['elevation_correlation'] = corr
        
        print(f"\n📐 Корреляция высоты и частоты шагов: {corr:.3f}")
        
        # Интерпретация корреляции
        if abs(corr) > 0.7:
            strength = "сильная"
        elif abs(corr) > 0.3:
            strength = "умеренная"
        else:
            strength = "слабая"
            
        direction = "положительная" if corr > 0 else "отрицательная"
        print(f"   • {strength.capitalize()} {direction} связь")
        print(f"   • Анализ основан на {len(df_elev)} точках")
    else:
        print("⚠️ Отсутствуют данные о высоте или частоте шагов")

    # 1.5 Самые популярные маршруты по периодам года
    if 'season' in df.columns and 'track_id' in df.columns:
        print("\n🏆 Анализ популярности маршрутов...")
        popular_tracks = df.groupby(['season', 'track_id']).size().reset_index(name='count')
        popular_tracks = popular_tracks.sort_values(['season', 'count'], ascending=[True, False])
        
        # Топ-3 трека в каждом сезоне
        top_tracks_by_season = popular_tracks.groupby('season').head(3).reset_index(drop=True)
        
        analysis_results['top_tracks'] = top_tracks_by_season.to_dict('records')
        
        # Создаем визуализацию
        fig = px.bar(
            top_tracks_by_season, 
            x='track_id', 
            y='count', 
            color='season',
            title='Самые популярные треки по сезонам (топ-3)',
            labels={'count': 'Количество точек', 'track_id': 'ID трека', 'season': 'Сезон'},
            facet_col='season', 
            facet_col_wrap=2,
            text='count'
        )
        
        fig.update_xaxes(tickangle=45, title_text="ID трека")
        fig.update_yaxes(title_text="Количество точек")
        fig.update_traces(textposition='outside')
        fig.update_layout(
            height=600,
            showlegend=True,
            legend_title="Сезон"
        )
        
        fig.show()
        
        print("\n🎯 Самые популярные треки по сезонам:")
        display(top_tracks_by_season)
        
        # Сводная статистика
        print(f"\n📋 Сводка по популярности:")
        for season in top_tracks_by_season['season'].unique():
            season_tracks = top_tracks_by_season[top_tracks_by_season['season'] == season]
            print(f"   • {season}: {len(season_tracks)} треков, максимальная активность: {season_tracks['count'].max()} точек")
    else:
        print("⚠️ Отсутствуют данные о сезонах или идентификаторах треков")

except Exception as e:
    print(f"\n❌ Ошибка при анализе трендов: {e}")
    print("Продолжаем выполнение с доступными данными...")

# Сохраняем результаты анализа
try:
    with open('trend_analysis_results.json', 'w', encoding='utf-8') as f:
        json.dump(analysis_results, f, ensure_ascii=False, indent=2)
    print("\n💾 Результаты анализа сохранены в trend_analysis_results.json")
except Exception as e:
    print(f"⚠️ Не удалось сохранить результаты анализа: {e}")

print("\n" + "=" * 60)
print("✅ АНАЛИЗ ТРЕНДОВ ЗАВЕРШЕН")
print("=" * 60)

In [ ]:
# ===== Часть 2: Разметка опасных участков (кластеризация) =====
print("\n⚠️ РАЗМЕТКА ОПАСНЫХ УЧАСТКОВ")
print("=" * 60)

from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Подготовка данных для кластеризации
print("🔧 Подготовка данных для кластеризации...")

# Выбираем признаки для кластеризации
clustering_features = ['lat', 'lon', 'elevation', 'temperature']
available_features = [f for f in clustering_features if f in df.columns]

if len(available_features) < 2:
    print("⚠️ Недостаточно признаков для кластеризации")
    # Добавляем случайные признаки для демонстрации
    df['lat_norm'] = np.random.randn(len(df))
    df['lon_norm'] = np.random.randn(len(df))
    available_features = ['lat_norm', 'lon_norm']

print(f"Используемые признаки: {available_features}")

# Нормализация
X = df[available_features].fillna(df[available_features].mean())
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Размер данных для кластеризации: {X_scaled.shape}")

# Функция для кластеризации с разными методами
def perform_clustering(X, method='dbscan', **params):
    """Выполняет кластеризацию выбранным методом"""
    if method == 'dbscan':
        eps = params.get('eps', 0.1)
        min_samples = params.get('min_samples', 10)
        model = DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean')
    elif method == 'kmeans':
        n_clusters = params.get('n_clusters', 4)
        model = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    elif method == 'agglomerative':
        n_clusters = params.get('n_clusters', 4)
        model = AgglomerativeClustering(n_clusters=n_clusters)
    else:
        raise ValueError(f"Неизвестный метод: {method}")
    
    labels = model.fit_predict(X)
    return labels, model

# Тестируем разные методы кластеризации
clustering_results = []

print("\n🔍 Тестирование методов кластеризации:")

# 1. DBSCAN с разными параметрами
for eps in [0.05, 0.1, 0.15, 0.2]:
    labels, model = perform_clustering(X_scaled, method='dbscan', eps=eps, min_samples=10)
    
    # Рассчитываем метрики только для нешумовых точек
    noise_mask = labels != -1
    if noise_mask.sum() > 1 and len(np.unique(labels[noise_mask])) > 1:
        sil_score = silhouette_score(X_scaled[noise_mask], labels[noise_mask])
        cal_score = calinski_harabasz_score(X_scaled[noise_mask], labels[noise_mask])
        db_score = davies_bouldin_score(X_scaled[noise_mask], labels[noise_mask])
    else:
        sil_score = cal_score = db_score = np.nan
    
    clustering_results.append({
        'method': 'DBSCAN',
        'params': f'eps={eps}',
        'n_clusters': len(np.unique(labels[noise_mask])) if noise_mask.sum() > 0 else 0,
        'noise_points': (labels == -1).sum(),
        'silhouette': sil_score,
        'calinski': cal_score,
        'davies_bouldin': db_score
    })

# 2. KMeans с разным количеством кластеров
for n_clusters in [3, 4, 5, 6, 7]:
    labels, model = perform_clustering(X_scaled, method='kmeans', n_clusters=n_clusters)
    
    sil_score = silhouette_score(X_scaled, labels)
    cal_score = calinski_harabasz_score(X_scaled, labels)
    db_score = davies_bouldin_score(X_scaled, labels)
    
    clustering_results.append({
        'method': 'KMeans',
        'params': f'k={n_clusters}',
        'n_clusters': n_clusters,
        'noise_points': 0,
        'silhouette': sil_score,
        'calinski': cal_score,
        'davies_bouldin': db_score
    })

# 3. Agglomerative Clustering
for n_clusters in [3, 4, 5]:
    labels, model = perform_clustering(X_scaled, method='agglomerative', n_clusters=n_clusters)
    
    sil_score = silhouette_score(X_scaled, labels)
    cal_score = calinski_harabasz_score(X_scaled, labels)
    db_score = davies_bouldin_score(X_scaled, labels)
    
    clustering_results.append({
        'method': 'Agglomerative',
        'params': f'k={n_clusters}',
        'n_clusters': n_clusters,
        'noise_points': 0,
        'silhouette': sil_score,
        'calinski': cal_score,
        'davies_bouldin': db_score
    })

# Создаем DataFrame с результатами
results_df = pd.DataFrame(clustering_results)
print("\n📊 Результаты кластеризации:")
display(results_df.sort_values('silhouette', ascending=False))

# Визуализация сравнения методов
fig = make_subplots(rows=2, cols=2,
                   subplot_titles=('Silhouette Score', 'Calinski-Harabasz Index',
                                  'Davies-Bouldin Index', 'Количество кластеров'),
                   vertical_spacing=0.15, horizontal_spacing=0.15)

colors = {'DBSCAN': 'blue', 'KMeans': 'red', 'Agglomerative': 'green'}

for method in results_df['method'].unique():
    method_data = results_df[results_df['method'] == method]
    
    fig.add_trace(go.Scatter(x=method_data['params'], y=method_data['silhouette'],
                            mode='lines+markers', name=f'{method} - Silhouette',
                            line=dict(color=colors.get(method, 'gray'))),
                  row=1, col=1)
    
    fig.add_trace(go.Scatter(x=method_data['params'], y=method_data['calinski'],
                            mode='lines+markers', name=f'{method} - Calinski',
                            line=dict(color=colors.get(method, 'gray'), dash='dash'),
                            showlegend=False),
                  row=1, col=2)
    
    fig.add_trace(go.Scatter(x=method_data['params'], y=method_data['davies_bouldin'],
                            mode='lines+markers', name=f'{method} - Davies-Bouldin',
                            line=dict(color=colors.get(method, 'gray'), dash='dot'),
                            showlegend=False),
                  row=2, col=1)
    
    fig.add_trace(go.Scatter(x=method_data['params'], y=method_data['n_clusters'],
                            mode='lines+markers', name=f'{method} - Clusters',
                            line=dict(color=colors.get(method, 'gray'), dash='dashdot'),
                            showlegend=False),
                  row=2, col=2)

fig.update_layout(height=800, title_text='Сравнение методов кластеризации', showlegend=True)
fig.update_xaxes(title_text='Параметры', row=2, col=1)
fig.update_xaxes(title_text='Параметры', row=2, col=2)
fig.update_yaxes(title_text='Score', row=1, col=1)
fig.update_yaxes(title_text='Index', row=1, col=2)
fig.update_yaxes(title_text='Index (меньше - лучше)', row=2, col=1)
fig.update_yaxes(title_text='Количество', row=2, col=2)

fig.show()


In [ ]:
# ===== Часть 3: Выбор лучшего метода и присвоение меток =====
print("\n🏆 ВЫБОР ЛУЧШЕГО МЕТОДА И ПРИСВОЕНИЕ МЕТОК")
print("=" * 60)

# Выбираем лучший метод на основе silhouette score
best_result = results_df.loc[results_df['silhouette'].idxmax()]
print(f"Лучший метод: {best_result['method']} с параметрами {best_result['params']}")
print(f"Silhouette Score: {best_result['silhouette']:.3f}")
print(f"Количество кластеров: {best_result['n_clusters']}")

# Применяем лучший метод
if best_result['method'] == 'DBSCAN':
    eps = float(best_result['params'].split('=')[1])
    labels, model = perform_clustering(X_scaled, method='dbscan', eps=eps, min_samples=10)
elif best_result['method'] == 'KMeans':
    n_clusters = int(best_result['params'].split('=')[1])
    labels, model = perform_clustering(X_scaled, method='kmeans', n_clusters=n_clusters)
else:
    n_clusters = int(best_result['params'].split('=')[1])
    labels, model = perform_clustering(X_scaled, method='agglomerative', n_clusters=n_clusters)

df['cluster_id'] = labels

# Анализ характеристик кластеров
print("\n📈 Анализ характеристик кластеров:")
cluster_stats = []

for cluster_id in np.unique(labels):
    cluster_mask = df['cluster_id'] == cluster_id
    cluster_data = df[cluster_mask]
    
    if cluster_id == -1:  # Шум (для DBSCAN)
        cluster_name = 'Шум'
    else:
        cluster_name = f'Кластер {cluster_id}'
    
    stats = {
        'cluster_id': cluster_id,
        'cluster_name': cluster_name,
        'size': len(cluster_data),
        'mean_elevation': cluster_data['elevation'].mean() if 'elevation' in df.columns else np.nan,
        'mean_temperature': cluster_data['temperature'].mean() if 'temperature' in df.columns else np.nan,
        'std_elevation': cluster_data['elevation'].std() if 'elevation' in df.columns else np.nan,
        'regions': ', '.join(cluster_data['region'].unique().astype(str)) if 'region' in df.columns else '',
        'terrain_types': ', '.join(cluster_data['terrain_type'].unique().astype(str)) if 'terrain_type' in df.columns else ''
    }
    
    cluster_stats.append(stats)

cluster_stats_df = pd.DataFrame(cluster_stats)
display(cluster_stats_df)

# Функция для присвоения меток риска и сложности эвакуации
def assign_risk_labels(df_with_clusters):
    """Присваивает метки риска и сложности эвакуации на основе характеристик кластеров"""
    
    df_labeled = df_with_clusters.copy()
    
    # Инициализируем колонки
    df_labeled['risk_level'] = 'Low'
    df_labeled['evacuation_difficulty'] = 'Easy'
    
    # Правила для определения рисков
    for cluster_id in df_labeled['cluster_id'].unique():
        cluster_mask = df_labeled['cluster_id'] == cluster_id
        cluster_data = df_labeled[cluster_mask]
        
        # 1. Определяем уровень риска
        if cluster_id == -1:  # Шумовые точки - высокий риск
            risk = 'High'
        elif len(cluster_data) < 5:  # Маленькие кластеры - средний риск
            risk = 'Medium'
        else:
            # Оцениваем по высоте и температуре
            mean_elev = cluster_data['elevation'].mean() if 'elevation' in cluster_data.columns else 0
            mean_temp = cluster_data['temperature'].mean() if 'temperature' in cluster_data.columns else 20
            
            risk_score = 0
            if mean_elev > 700:
                risk_score += 2  # Высокогорье - высокий риск
            elif mean_elev > 300:
                risk_score += 1  # Средняя высота - средний риск
            
            if mean_temp < -5:
                risk_score += 2  # Сильный мороз - высокий риск
            elif mean_temp < 5:
                risk_score += 1  # Холодно - средний риск
            
            # Определяем уровень риска по score
            if risk_score >= 3:
                risk = 'High'
            elif risk_score >= 1:
                risk = 'Medium'
            else:
                risk = 'Low'
        
        # 2. Определяем сложность эвакуации
        if risk == 'High':
            evac = 'Hard'
        elif risk == 'Medium':
            evac = 'Medium'
        else:
            # Для низкого риска оцениваем по удаленности от дорог
            if 'terrain_type' in cluster_data.columns:
                terrains = cluster_data['terrain_type'].value_counts()
                if 'горы' in terrains.index and terrains['горы'] > len(cluster_data) * 0.5:
                    evac = 'Hard'
                elif 'холмы' in terrains.index and terrains['холмы'] > len(cluster_data) * 0.5:
                    evac = 'Medium'
                else:
                    evac = 'Easy'
            else:
                evac = 'Easy'
        
        # Присваиваем метки
        df_labeled.loc[cluster_mask, 'risk_level'] = risk
        df_labeled.loc[cluster_mask, 'evacuation_difficulty'] = evac
    
    return df_labeled

# Присваиваем метки
df_labeled = assign_risk_labels(df)

print("\n✅ Метки присвоены. Распределение:")
print(f"• Уровни риска: {df_labeled['risk_level'].value_counts().to_dict()}")
print(f"• Сложность эвакуации: {df_labeled['evacuation_difficulty'].value_counts().to_dict()}")

# Визуализация распределения меток
fig = make_subplots(rows=1, cols=2, 
                   subplot_titles=('Распределение уровней риска', 'Сложность эвакуации'),
                   specs=[[{'type': 'pie'}, {'type': 'pie'}]])

risk_counts = df_labeled['risk_level'].value_counts()
evac_counts = df_labeled['evacuation_difficulty'].value_counts()

colors_risk = {'High': 'red', 'Medium': 'orange', 'Low': 'green'}
colors_evac = {'Hard': 'red', 'Medium': 'orange', 'Easy': 'green'}

fig.add_trace(go.Pie(labels=risk_counts.index, values=risk_counts.values,
                     marker=dict(colors=[colors_risk.get(l, 'gray') for l in risk_counts.index]),
                     hole=0.3, name='Уровни риска'),
              row=1, col=1)

fig.add_trace(go.Pie(labels=evac_counts.index, values=evac_counts.values,
                     marker=dict(colors=[colors_evac.get(l, 'gray') for l in evac_counts.index]),
                     hole=0.3, name='Сложность эвакуации'),
              row=1, col=2)

fig.update_layout(title_text='Распределение меток риска и эвакуации', height=500)
fig.show()


In [ ]:
# ===== Часть 4: Визуализация кластеров на карте (исправлено и дополнено) =====
import os

# Загружаем переменные окружения
from dotenv import load_dotenv
load_dotenv()

# Определяем папку для отчётов
report_dir = os.getenv("REPORT_DIR", "report")
os.makedirs(report_dir, exist_ok=True)  # создаёт папку, если её нет
try:
    from folium.plugins import HeatMap
except ImportError as e:
    st.error("❌ Не хватает folium.plugins. Установите: pip install folium")
    raise e

print("\n🗺️ ВИЗУАЛИЗАЦИЯ КЛАСТЕРОВ НА КАРТЕ")
print("=" * 60)

# Центр карты
center_lat = df_labeled['lat'].mean()
center_lon = df_labeled['lon'].mean()

# Карта с рисками (цветные маркеры)
m = folium.Map(location=[center_lat, center_lon], zoom_start=8, control_scale=True)
risk_colors = {'High': 'red', 'Medium': 'orange', 'Low': 'green'}

for _, row in df_labeled.iterrows():
    color = risk_colors.get(row['risk_level'], 'gray')
    popup_text = f"""
    <b>ID:</b> {row.get('id', 'N/A')}<br>
    <b>Трек:</b> {row.get('track_id', 'N/A')}<br>
    <b>Риск:</b> <span style='color:{color}'>{row.get('risk_level', 'N/A')}</span><br>
    <b>Высота:</b> {row.get('elevation', 'N/A'):.0f} м<br>
    <b>Температура:</b> {row.get('temperature', 'N/A'):.1f}°C
    """
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=folium.Popup(popup_text, max_width=300),
        tooltip=f"Риск: {row['risk_level']} | {row['elevation']:.0f}м"
    ).add_to(m)

# Легенда
legend_html = '''
<div style="position: fixed; bottom: 50px; left: 50px; width: 180px; height: 130px; 
     border:2px solid grey; z-index:9999; font-size:14px; background-color:white; opacity:0.9; padding: 10px;">
     <b>Легенда:</b><br>
     ● <span style="color:red">Высокий риск</span><br>
     ● <span style="color:orange">Средний риск</span><br>
     ● <span style="color:green">Низкий риск</span>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Сохраняем карту рисков
map_filename = os.path.join(report_dir, 'cluster_risk_map.html')
m.save(map_filename)
print(f"✅ Карта сохранена: {map_filename}")
display(m)

# Тепловая карта (HeatMap)
try:
    m_heat = folium.Map(location=[center_lat, center_lon], zoom_start=8)
    heat_data = [
        [row['lat'], row['lon'], 1.0 if row['risk_level'] == 'High' else 0.5]
        for _, row in df_labeled.iterrows()
    ]
    HeatMap(heat_data, radius=15, blur=10, gradient={0.4: 'blue', 0.6: 'lime', 1.0: 'red'}).add_to(m_heat)
    
    heatmap_filename = os.path.join(report_dir, 'risk_heatmap.html')
    m_heat.save(heatmap_filename)
    print(f"✅ Тепловая карта сохранена: {heatmap_filename}")
except Exception as e:
    print(f"⚠️ Не удалось построить тепловую карту: {e}")

In [ ]:
# ===== Часть 5: Сохранение результатов (исправлено) =====
print("\n💾 СОХРАНЕНИЕ РЕЗУЛЬТАТОВ")
print("=" * 60)

# Создаём папку report, если её нет
report_dir = os.getenv("REPORT_DIR", "report")
os.makedirs(report_dir, exist_ok=True)

# 5.1 Сохраняем размеченный датасет
output_filename = os.path.join(report_dir, 'labeled_dataset.csv')
df_labeled.to_csv(output_filename, index=False, encoding='utf-8')
print(f"✅ Размеченный датасет сохранён: {output_filename}")

# 5.2 Отчёт кластеризации
clustering_report = {
    'best_method': best_result['method'],
    'best_params': best_result['params'],
    'silhouette_score': float(best_result['silhouette']),
    'calinski_score': float(best_result['calinski']),
    'davies_bouldin_score': float(best_result['davies_bouldin']),
    'n_clusters': int(best_result['n_clusters']),
    'n_noise_points': int(df_labeled[df_labeled['cluster_id'] == -1].shape[0]) if -1 in df_labeled['cluster_id'].values else 0,
    'risk_distribution': df_labeled['risk_level'].value_counts().to_dict(),
    'evacuation_distribution': df_labeled['evacuation_difficulty'].value_counts().to_dict(),
    'timestamp': pd.Timestamp.now().isoformat()
}

report_path = os.path.join(report_dir, 'clustering_report.json')
with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(clustering_report, f, ensure_ascii=False, indent=2)
print(f"✅ Отчёт кластеризации сохранён: {report_path}")

# 5.3 Статистика по кластерам
cluster_stats_df.to_csv(os.path.join(report_dir, 'cluster_statistics.csv'), index=False, encoding='utf-8')
print("✅ Статистика по кластерам сохранена")

# 5.4 Сводный отчёт
summary_report = f"""# ОТЧЕТ ПО МОДУЛЮ Б\n\n## Общая информация\n- Дата: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n- Записей: {len(df_labeled)}\n- Уникальных треков: {df_labeled['track_id'].nunique()}\n\n## Лучший метод: {best_result['method']} (score: {best_result['silhouette']:.3f})\n\n## Распределение рисков\n"""
for risk, count in df_labeled['risk_level'].value_counts().items():
    summary_report += f"- {risk}: {count} ({count/len(df_labeled)*100:.1f}%)\n"

summary_report += "\n## Выводы\n1. ✅ Анализ завершён\n2. ✅ Данные готовы к передаче в Модуль В"

summary_path = os.path.join(report_dir, 'module_b_summary_report.md')
with open(summary_path, 'w', encoding='utf-8') as f:
    f.write(summary_report)
print(f"✅ Сводный отчёт сохранён: {summary_path}")

# 5.5 Предпросмотр
print("\n🔍 ПРЕДПРОСМОТР:")
display(df_labeled[['track_id', 'region', 'lat', 'lon', 'risk_level', 'evacuation_difficulty']].head(10))

print("\n✅ МОДУЛЬ Б УСПЕШНО ЗАВЕРШЕН!")
print("=" * 60)